In [1]:
import pandas as pd


from constants import okr_excel_path, researcher_info_excel_path, salesperson_info_excel_path, output_excel_path
from utils import get_df_roadshow, get_sales_name_list, get_sales_name_list, write_dfs_to_excel
from utils import print_na_rate


from constants import SHEET_NAMES


In [2]:
df_okr = pd.read_excel(okr_excel_path, sheet_name='路演')
df_researcher_info = pd.read_excel(researcher_info_excel_path, usecols='A:C')
df_salespeople_info =  pd.read_excel(salesperson_info_excel_path)

In [3]:
sales_name_list = get_sales_name_list(df_salespeople_info)

In [4]:
df_roadshow, df_special = get_df_roadshow(df_okr, sales_name_list)

In [5]:
df_roadshow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3641 entries, 0 to 3640
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   序号      3641 non-null   int64 
 1   服务事项    3641 non-null   object
 2   客户机构    3641 non-null   object
 3   客户分级    3641 non-null   object
 4   客户区域    3641 non-null   object
 5   所属团队    3641 non-null   object
 6   研究员     3641 non-null   object
dtypes: int64(1), object(6)
memory usage: 199.2+ KB


In [6]:
dfs_dict = {name[3:]: globals()[name] for name in SHEET_NAMES if name in globals()}

In [7]:

# Now you can use dfs_dict with the write_dfs_to_excel function
write_dfs_to_excel(output_excel_path, dfs_dict)


In [8]:
# make above all the codes into a pipeline
# Path: okr-eval-v3.ipynb
import pandas as pd


In [9]:
dfs_dict.keys()

dict_keys(['roadshow', 'special'])

In [10]:
# print the rows with NAN value in df_roadshow
df_row_nan = df_roadshow[df_roadshow.isnull().T.any()]

In [11]:
df_row_nan

,序号,服务事项,客户机构,客户分级,客户区域,所属团队,研究员
